In [ ]:
from operator import index

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.power import TTestIndPower
import re

In [ ]:
station = pd.read_csv("./094/station.csv", sep='\t', engine="python")
patient = pd.read_csv("./094/patient.csv", sep='\t', engine="python")
observation = pd.read_csv("./094/observation.csv", sep='\t', engine="python")

## 2.1 A

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = observation['oximetry'].values
x = observation.drop(columns=['oximetry'], axis=1).values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# train_df = X_train.copy()
# train_df["oximetry"] = y_train
# 
# test_df = X_test.copy()
# test_df["oximetry"] = y_test
# 
# train_df.to_csv("train_raw.csv", index=False)
# test_df.to_csv("test_raw.csv", index=False)

## 2.1 B

In [ ]:
train_df = pd.read_csv("train_raw.csv")

In [139]:
train_df.dtypes

SpO₂                     float64
HR                       float64
PI                       float64
RR                       float64
EtCO₂                    float64
FiO₂                     float64
PRV                      float64
BP                       float64
Skin Temperature         float64
Motion/Activity index    float64
PVI                      float64
Hb level                 float64
SV                       float64
CO                       float64
Blood Flow Index         float64
PPG waveform features    float64
Signal Quality Index     float64
Respiratory effort       float64
O₂ extraction ratio      float64
SNR                      float64
latitude                 float64
longitude                float64
oximetry                 float64
dtype: object

In [ ]:
train_df.isnull().sum().sum()

In [ ]:
train_df.duplicated().sum()

## 2.1 C

In [ ]:
from scipy.stats import skew

skews = skew(x, axis=0)

skew_df = pd.DataFrame({
    'Feature': x.columns,
    'Skewness': skews
})

skew_df

Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler 

scaler = MinMaxScaler()

X_mm = scaler.fit_transform(X_train) 

X_mm

In [ ]:
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler()

X_std = scaler.fit_transform(X_train) 

X_std

In [144]:
import numpy as np

print("StandardScaler:")
print("Среднее =", np.mean(X_train, axis=0).round(3))
print("Ст. отклонение =", np.std(X_train, axis=0).round(3))

print("\nMinMaxScaler:")
print("Минимум =", np.min(X_train, axis=0).round(3))
print("Максимум =", np.max(X_train, axis=0).round(3))

StandardScaler:
Среднее = [ 97.541  79.383   8.724  16.139  39.649  59.381 117.756 104.438  35.982
  10.948  15.311  14.717  80.903   4.099  45.595  47.458  46.789  49.718
   0.249  29.965  29.998   7.55 ]
Ст. отклонение = [6.0600e-01 4.9700e+00 3.4960e+00 1.0360e+00 1.4290e+00 1.0745e+01
 2.1867e+01 4.0880e+00 8.4600e-01 1.4470e+00 1.5830e+00 7.1900e-01
 4.9820e+00 1.3500e-01 1.2267e+01 1.1966e+01 1.2199e+01 1.4054e+01
 2.9000e-02 5.7590e+00 2.2823e+01 7.3843e+01]

MinMaxScaler:
Минимум = [  95.      60.       0.46    12.      35.194   21.      36.951   90.
   33.       5.736   10.      12.059   60.       4.       1.091    0.
    0.       0.       0.2     20.     -54.8   -156.474]
Максимум = [ 99.791 100.     18.718  20.     45.    100.    198.724 120.     38.
  17.137  20.     18.    100.      8.    100.    100.     99.353  98.785
   0.3    40.     68.798 178.004]


In [146]:
import numpy as np

print("StandardScaler:")
print("Среднее =", np.mean(X_std, axis=0).round(3))
print("Ст. отклонение =", np.std(X_std, axis=0).round(3))

print("\nMinMaxScaler:")
print("Минимум =", np.min(X_mm, axis=0).round(3))
print("Максимум =", np.max(X_mm, axis=0).round(3))

StandardScaler:
Среднее = [-0. -0.  0. -0.  0. -0. -0. -0. -0.  0. -0.  0. -0.  0.  0.  0. -0. -0.
  0. -0.  0.  0.]
Ст. отклонение = [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]

MinMaxScaler:
Минимум = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Максимум = [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


Transformers

In [ ]:
from sklearn.preprocessing import PowerTransformer
from matplotlib import pyplot

pyplot.hist(X_train[0], bins=10)

power = PowerTransformer(method='yeo-johnson', standardize=True) 
X_pt = power.fit_transform(X_train)

pyplot.hist(X_pt[0], bins=10) 

In [ ]:
from sklearn.preprocessing import QuantileTransformer
from matplotlib import pyplot

pyplot.hist(X_train[0], bins=10)

power = QuantileTransformer(output_distribution='normal', random_state=42)
X_qt = power.fit_transform(X_train)

pyplot.hist(X_qt[0], bins=10)